## 決策樹

In [ ]:
#準備引入存在google drive上的資料夾
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#引入蝦皮資料集
import re
import pandas as pd
try:
    df = pd.read_csv("/content/drive/MyDrive/shopee_item_category.csv")
    # Continue processing the content
except UnicodeDecodeError as e:
    # Handle the UnicodeDecodeError by specifying a different encoding or replacing invalid characters
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print("File not found. Check the file path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
#引入前處理過的蝦皮資料集商品名稱
df1 = pd.read_csv("/content/drive/MyDrive/output.csv")
#將兩份資料集合併，方便之後提取前處理過的商品名稱和原有的大類、小類欄位
merged_df = pd.concat([df, df1], axis=1)
data=merged_df

In [ ]:
#引入需要的套件
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
#各訓練10次大類、小類的決策樹分類模型，一次隨機取15000筆資料
for i in range(10):
  classified_data = data.sample(n=15000)
  classified_data['name'] = data['name'].fillna('')
  tfidf_vectorizer = TfidfVectorizer()
  X = tfidf_vectorizer.fit_transform(classified_data['name'])

# 標籤編碼
  label_encoder_big = LabelEncoder()
  label_encoder_small = LabelEncoder()

  y_big = label_encoder_big.fit_transform(classified_data['大類'])
  y_small = label_encoder_small.fit_transform(classified_data['小類'])

# 分割資料集
  X_train, X_test, y_train_big, y_test_big, y_train_small, y_test_small = train_test_split(
      X, y_big, y_small, test_size=0.2, random_state=42
  )
#大類訓練測試
  clf_big = DecisionTreeClassifier()
  clf_big.fit(X_train, y_train_big)
  pred=clf_big.predict(X_test)
  zipped_arrays = zip(y_test_big, pred)
#小類訓練測試
  clf_small = DecisionTreeClassifier()
  clf_small.fit(X_train, y_train_small)
  pred1=clf_small.predict(X_test)
  zipped_arrays1 = zip(y_test_small, pred1)
#計算測試出來的正確率
  common_elements_count = sum(x == y for x, y in zipped_arrays)
  common_elements_count1 = sum(x == y for x, y in zipped_arrays1)
  print("大類準確率",common_elements_count/pred.size,"小類準確率",common_elements_count1/pred1.size)

大類準確率 0.5913333333333334 小類準確率 0.30466666666666664
大類準確率 0.6006666666666667 小類準確率 0.288
大類準確率 0.6163333333333333 小類準確率 0.31
大類準確率 0.601 小類準確率 0.306
大類準確率 0.6023333333333334 小類準確率 0.31533333333333335
大類準確率 0.6053333333333333 小類準確率 0.323
大類準確率 0.6073333333333333 小類準確率 0.31466666666666665
大類準確率 0.599 小類準確率 0.2966666666666667
大類準確率 0.5993333333333334 小類準確率 0.31266666666666665
大類準確率 0.6283333333333333 小類準確率 0.33166666666666667


10次共執行15分鐘
大類準確率平均約6成
小類準確率平均約3成

In [ ]:
# 讀入未分類的資料集（name經前處理
unclassified_data = pd.read_csv("/content/drive/MyDrive/output3.csv")  # 替換為實際的檔案路徑data_a = pd.read_csv("/content/drive/MyDrive/output3.csv")
#去除有缺失值的資料
cleaned_data = unclassified_data.dropna(subset=['name']).copy()

In [ ]:
#預測大類
X_unclassified = tfidf_vectorizer.transform(cleaned_data['name'])
predictions_big = clf_big.predict(X_unclassified)
#預測小類
X_unclassified_small = tfidf_vectorizer.transform(cleaned_data['name'])
predictions_small = clf_small.predict(X_unclassified)

In [ ]:
# 將預測結果轉換回原始的類別名稱
predicted_big_category = label_encoder_big.inverse_transform(predictions_big)
predicted_small_category = label_encoder_small.inverse_transform(predictions_small)

# 將預測結果添加到未分類的商品資料集
#cleaned_data['預測大類'] = predicted_big_category
#cleaned_data['預測小類'] = predicted_small_category
# 在原始資料框上使用 .loc 賦值，避免SettingWithCopyWarning
cleaned_data.loc[:, '預測大類'] = predicted_big_category
cleaned_data.loc[:, '預測小類'] = predicted_small_category

# 保存結果
cleaned_data.to_csv('cleaned_data_with_predictions_decision_tree.csv', index=False)


In [ ]:
c=pd.read_csv('cleaned_data_with_predictions_decision_tree.csv')

In [ ]:
c[457:460]

,Unnamed: 0,name,預測大類,預測小類
457,457,日本浴室除霉,書籍及雜誌期刊,其他
458,458,NIKE DUNK 白銀斷勾DJ,男鞋,休閒鞋款
459,459,博士清潔錠,書籍及雜誌期刊,其他


In [ ]:
c[674:675]

,Unnamed: 0,name,預測大類,預測小類
674,675,純水柔濕巾,書籍及雜誌期刊,濕紙巾


以上部分資料為例：

大類多被分到書籍、小類多被分到其他

但也有少數分對大or小類甚至全對

整體而言決策樹模型分類的結果不佳


## BERT

In [ ]:
! pip install -U ckip-transformers
# 安裝 ckip-transformers 庫
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 從 ckip_transformers 庫導入版本信息
from ckip_transformers import __version__
# 導入 ckp_transformers 庫中的 NLP 相關模塊，包括斷詞、詞性標記和命名實體識別
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker


# Show version
print(__version__)

# 初始化驅動程式
# 初始化斷詞驅動程式，使用 "albert-base" 模型
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="albert-base")
# 初始化詞性標記驅動程式，使用 "albert-base" 模型
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="albert-base")
# 初始化命名實體識別驅動程式，使用 "albert-base" 模型
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="albert-base")
print("Initializing drivers ... all done")
print()

0.3.4
Initializing drivers ... WS


Initializing drivers ... POS


Initializing drivers ... NER


Initializing drivers ... all done



In [ ]:
try:
    df = pd.read_excel("/content/drive/Shareddrives/colab/Jambo_3.xlsx")
    # Continue processing the content
except UnicodeDecodeError as e:
    # Handle the UnicodeDecodeError by specifying a different encoding or replacing invalid characters
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print("File not found. Check the file path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

File not found. Check the file path.


In [ ]:
name = []
for i in df['商品名稱']:
    name.append(i)
print(name)

['Q MAN - Pokemon 寶可夢 小火龍的火鍋店積木', 'essence 艾森絲 持久遮瑕膏(7ml) 自然膚色／淺膚色【小三美日】D473151', '威剛 XPG GAMMIX S50LiteCS 512GB M.2 SSD PCIE Gen4 x4 現貨 廠商直送', '圖解國富論{新版}2019[88折] 11100879625', 'Case Logic JAUNT 15.6吋筆電後背包 石墨黑 WMBP-115', 'H&D東稻家居︱夢露灰色收納三人沙發/沙發床【22JS1/402-2】', 'Super wings超級飛俠 變形桑妮 玩具反斗城', 'NOW健而婷 螯合鋅植物膠囊 100顆/瓶 廠商直送 大樹', '【J’code 真愛密碼】無可取代鋼項鍊/男款', 'CACO-MIT 滑板臘腸狗上衣-女【D2CA018】', '【正心堂】黃金糖 水晶糖 黃金糖 晶糖 金糖 送客糖 喜糖 迎賓糖 麥芽糖 糖果 婚禮糖果 古早味 古早味糖果', 'New Balance 408 系列 男女 休閒鞋 白銀 ML408C-D', 'KENYU 便利尿袋 3入 塞車/休閒可用 3KP-100', '新約魔法禁書目錄(12)', 'LADY 魔力V系列 素面無痕機能調整型成套內衣 E-F罩 三角 平口 隨機配褲 ( 輕巧膚 )', '馨乃樂 cimilre PP標準小奶瓶(2入)', 'Dr.HOLI 全方位保健起司泥 起司泥 寵物 零食 牛磺酸 乳酸菌 坡尿酸 營養補充 低脂 低熱量 無乳糖', 'HOLA 葡萄牙純棉緹花毛巾踏墊50x80cm 框紋深藍', '【德國WMF】PROFI SELECT 不沾煎鍋28CM+瀝油鍋鏟', '海淨 純水濕紙巾20張x120包/箱(四款可選) 純水/花語/綠茶/嬰兒 宅配免運', '【MOSS CLUB】假兩件 圓點 女 短袖 襯衫 藍色 黃色 日系 寬鬆 休閒 亮眼 點點 波點 V領 春夏 弧形衣襬', 'OLAY滋潤淡紋優惠組（活膚精華霜50GX2+多元修護晚霜14G）', 'Snatch 銀球糖果圈圈手作耳夾 - 粉藍', 'Levis 512低腰修身窄管牛仔褲 上寬下窄/中藍基本款 /恆溫調節機能 /彈性布料 男-熱賣單品28833-0442', '【3M】 台灣製造 模型工具 研磨砂紙 水砂紙 3張

In [ ]:
# 安裝並導入 ckiptagger 函式庫
!pip install ckiptagger
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
# 下載和解壓縮模型資料，這些模型用於中文自然語言處理
# 下載至 ./data.zip (2GB) 然後解壓縮至 ./data/
# data_utils.download_data_url("./") # iis-ckip
data_utils.download_data_gdown("./") # gdrive-ckip
# 設定模型資料路徑，假設 "data" 文件夾位於當前工作目錄中
model_dir = './data'
# 初始化分詞（WS）、詞性標記（POS）和命名實體識別（NER）模型
ws = WS(model_dir)
pos = POS(model_dir)
ner = NER(model_dir)

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:35<00:00, 52.1MB/s]
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equ

In [ ]:
for i in range(len(name)):
  name[i] = re.sub(u"\（.*?\）",'',name[i])
  name[i] = re.sub(u"\(.*?\)",'',name[i])
  name[i] = re.sub(u"\【.*?\】",'',name[i])
  name[i] = re.sub(r'\*.*','',name[i])
  name[i] = re.sub(r'-','',name[i])
  name[i] = re.sub(r'公主\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'公主派對','',name[i])
  name[i] = re.sub(r'PRINCESS PARTY','',name[i])
  name[i] = re.sub(r'Princess Party','',name[i])
  name[i] = re.sub(r'叫賣\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'舊三郎\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'賦哥\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'畢哥\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'大師兄\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'新三郎\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'秀炮\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'大蘋\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'王三郎\s{1,2}\d{4}\s{1,2}[A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'\d{3} \d{4} [A-Z]\d{2}','',name[i])
  name[i] = re.sub(r'第\s{0,1}\d{3}\s{0,1}標','',name[i])

print(name)

['Q MAN  Pokemon 寶可夢 小火龍的火鍋店積木', 'essence 艾森絲 持久遮瑕膏 自然膚色／淺膚色D473151', '威剛 XPG GAMMIX S50LiteCS 512GB M.2 SSD PCIE Gen4 x4 現貨 廠商直送', '圖解國富論{新版}2019[88折] 11100879625', 'Case Logic JAUNT 15.6吋筆電後背包 石墨黑 WMBP115', 'H&D東稻家居︱夢露灰色收納三人沙發/沙發床', 'Super wings超級飛俠 變形桑妮 玩具反斗城', 'NOW健而婷 螯合鋅植物膠囊 100顆/瓶 廠商直送 大樹', '無可取代鋼項鍊/男款', 'CACOMIT 滑板臘腸狗上衣女', '黃金糖 水晶糖 黃金糖 晶糖 金糖 送客糖 喜糖 迎賓糖 麥芽糖 糖果 婚禮糖果 古早味 古早味糖果', 'New Balance 408 系列 男女 休閒鞋 白銀 ML408CD', 'KENYU 便利尿袋 3入 塞車/休閒可用 3KP100', '新約魔法禁書目錄', 'LADY 魔力V系列 素面無痕機能調整型成套內衣 EF罩 三角 平口 隨機配褲 ', '馨乃樂 cimilre PP標準小奶瓶', 'Dr.HOLI 全方位保健起司泥 起司泥 寵物 零食 牛磺酸 乳酸菌 坡尿酸 營養補充 低脂 低熱量 無乳糖', 'HOLA 葡萄牙純棉緹花毛巾踏墊50x80cm 框紋深藍', 'PROFI SELECT 不沾煎鍋28CM+瀝油鍋鏟', '海淨 純水濕紙巾20張x120包/箱 純水/花語/綠茶/嬰兒 宅配免運', '假兩件 圓點 女 短袖 襯衫 藍色 黃色 日系 寬鬆 休閒 亮眼 點點 波點 V領 春夏 弧形衣襬', 'OLAY滋潤淡紋優惠組', 'Snatch 銀球糖果圈圈手作耳夾  粉藍', 'Levis 512低腰修身窄管牛仔褲 上寬下窄/中藍基本款 /恆溫調節機能 /彈性布料 男熱賣單品288330442', ' 台灣製造 模型工具 研磨砂紙 水砂紙 3張入 800番 粗', '練習說再見療癒雙書：星空下的等待+小黑與櫻花[75折]11100916322', '茶樹莊園茶樹檸檬超濃縮洗碗精', 'YAMAHA BWS bws水冷 bws七期 專用 Gozilla 越野風 方向燈 尾燈 防護飾罩 方向燈罩

In [ ]:
print(df)

              channel_name     商品id  \
0               279-🛒現貨直播車  5938153   
1      TR Box寶藏屋：傘的專家、居的職人  4558966   
2                     阿清服裝  3244775   
3      TR Box寶藏屋：傘的專家、居的職人  6213367   
4      TR Box寶藏屋：傘的專家、居的職人  5088889   
...                    ...      ...   
18747      邦成-自倉（C倉出貨付款全開）  3142095   
18748      邦成-自倉（C倉出貨付款全開）  3572237   
18749      邦成-自倉（C倉出貨付款全開）  5221207   
18750      邦成-自倉（C倉出貨付款全開）  5591819   
18751      邦成-自倉（C倉出貨付款全開）  3198935   

                                                    商品名稱  
0                                  第 014 標 (1/9)多款食品(隨機)  
1      第 051 標(9/8)台灣製隨機室內拖鞋(顏色款式隨機)-23&24&25&26(號)-尺...  
2                                    Adidas三葉草三線短褲(9874)  
3               第208標(2/5)限宅配：2022年6月恐龍餅/口味隨機/120G一袋149元  
4      第 104 標(10/27)55度恒溫保暖杯座-約22.3X14.3X18.8cm-重量25...  
...                                                  ...  
18747  叫賣 0425 A05-(4罐)【紅薏仁】超臨界毛孔美白露 「150ML」(效期2022/9...  
18748         畢哥 0609 A17- Princess Party 純水濕紙巾(80抽/包)*1  


In [ ]:
# 使用 ckiptagger 中的分詞（WS）模型來處理名稱（name）
word_sentence_list = ws(name)
# 使用詞性標記（POS）模型來處理分詞結果
pos_sentence_list = pos(word_sentence_list)
# 使用命名實體識別（NER）模型來處理分詞結果和詞性標記結果
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [ ]:
temp = []
# 定義一個函式，用於列印分詞和詞性標記
def print_word_pos_sentence(word_sentence, pos_sentence, target_pos):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
      print(f"{word}({pos})", end="\u3000")
      temp.append(f"{word}({pos})")
    print()
    return
target_pos = ["Na"]
for i, sentence in enumerate(name):
    print()
    print(f"'{sentence}'")
     # 呼叫 print_word_pos_sentence 函式，將分詞、詞性標記傳遞給函式
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i], target_pos)
     # 遍歷並列印命名實體
    for entity in sorted(entity_sentence_list[i]):
        print(entity)


NameError: ignored

Neu	數詞定詞/
Nf	量詞/
PARENTHESISCATEGORY	括號/
D	副詞/
Nd	時間詞/
Nb	專有名詞/
VK	狀態句賓動詞/
Neqa	數量定詞/
PERIODCATEGORY	句號/
QUESTIONCATEGORY	問號/
SEMICOLONCATEGORY	分號/
DASHCATEGORY	破折號/
Ncd	位置詞/
Da	數量副詞/
Dfa	動詞前程度副詞/
Caa	對等連接詞/
VJ	狀態及物動詞/
VCL	動作接地方賓語動詞/
P	介詞
ETCCATEGORY	刪節號
COLONCATEGORY	冒號

In [ ]:
def newName(word_sentence, pos_sentence, target_pos):
    new_name = ""
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        if pos not in target_pos:
          new_name = new_name+word

    return new_name

target_pos = ["Nf"] + ["Neu"] + ["PARENTHESISCATEGORY"] + ["D"] + ["Nd"] + ["VK"] + ["Neqa"] + ["PERIODCATEGORY"]	+ ["QUESTIONCATEGORY"] + ["SEMICOLONCATEGORY"] + ["DASHCATEGORY"] + ["Ncd"] + ["Da"] + ["Caa"] + ["Dfa"] + ["VJ"] + ["VCL"] + ["P"] + ["ETCCATEGORY"] + ["COLONCATEGORY"] # 指定不要要印出的詞性
new_names = []
for i, sentence in enumerate(name):
    # 調用 newName 函式，將分詞、詞性標記和目標詞性傳遞給函式
    new_names.append(newName(word_sentence_list[i],  pos_sentence_list[i], target_pos))
print(new_names)

[' 食品', '台灣製室拖鞋尺寸擇', 'Adidas三葉草短褲', '限宅配恐龍餅/口味/G', '恒溫保暖杯座XXcm重量G', ' 女神膠原蛋白飲', 'ILZ薑黃B+C', 'DKNY外套灰/綠/黑', '社團 Columbia 背幾何黑短EM', '/KANGOL女性護理清洗液洋/洋甘菊綠/綠茶莓/蔓越莓擇', '可愛趴趴動物吸水地墊xCM', 'Ajuste 愛伽絲高效防曬噴霧', 'BIONAP百貝 克痠騰膠囊', '韓國GOODAL青橘能量眼膜', '日本竹炭牙線', 'G7三合溶咖啡Gx包入', 'Moschino Toy Boy男淡香精', '台灣製雙面泡棉膠/寬CM//', ' GUESS 馬卡龍色厚磅大學T', 'SODA BEAUTY炭酸痘潔顏慕絲 ', '可掛式防潮除濕袋G', '台灣製不銹鋼唐老鴨氣噗噗鴨笛音壺/xCM/3L/', '強效升級 管道疏通劑', 'MAYORA 起司威化餅乾G', 'JIHTENGInPod3多功能藍芽耳機', '迷你折疊刀', '公主 嬰兒純水柔濕巾', ' GUESS 背心', '銀離子除臭噴霧', '日本FaFa熊寶貝洗衣粉Baby花香 ', '折疊垃圾桶', '采妍益生菌酵母軟糖', '加厚束口家務手套/顏色/', 'PRIL 高效能洗碗精 ', 'LEERFEI木紋藍芽喇叭', 'AJ白黑藍DC', '萬毛牙刷', '鴕鳥精膠囊 ', 'Himalaya 喜馬拉雅阿育吠陀舒緩膏', "Cab's 苦瓜胜肽 ", 'THBPARUTE帕樂德英式煎捲餅乾禮盒', '乾拌麵綜合包', " 韓國S'MORE OHGODMALLOW 棉花糖脆餅", ' GUESS 毛絨問號大學T', '防水防滑防寒手套', 'Sebamed 私密護潔露', 'BRE 雙專利磷蝦魚油軟膠囊', '吸水浴帽/顏色/', 'TYI洗顏專科超微米潔顏乳 120', 'FC日本味王療肺草EX軟膠囊', '麻辣鴨血', '吸盤摺疊化妝鏡', '  LED燈泡', 'RIPNDIP外太空短T黑', '風獅爺一條根電氣石貼布熱/涼擇', '英YARDLEY雅麗高雅 體香劑', 'DUNK黑綠W CW', '桌面升降手機支架', ' 運動鞋清潔濕巾', 'ASEDO 台灣亞斯多石墨烯臭除菌科技短襪', '手機支架/顏色/', '日系復古純

In [ ]:
resultDf = pd.DataFrame()
resultDf = resultDf.assign(name=new_names)
print(resultDf)

NameError: ignored

In [ ]:
from google.colab import files

In [ ]:
resultDf.to_csv('output.csv', encoding = 'utf-8-sig')
files.download('output.csv')

NameError: ignored

In [ ]:
! pip install transformers
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")
print(tokenizer.tokenize('哈嘍你好嗎衷心感謝'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.4 MB/s eta 0:00:00


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['哈', '嘍', '你', '好', '嗎', '衷', '心', '感', '謝']


In [ ]:
example_text = "哈嘍你好嗎衷心感謝"
bert_input = tokenizer(example_text,padding='max_length',
                       max_length = 10,
                       truncation=True,
                       return_tensors="pt")
# ------- bert_input ------
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101, 1506, 1651,  872, 1962, 1621, 6141, 2552, 2697,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
import re
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
try:
    df = pd.read_csv("/content/drive/Shareddrives/colab/shopee_item_category.csv")
    # Continue processing the content
except UnicodeDecodeError as e:
    # Handle the UnicodeDecodeError by specifying a different encoding or replacing invalid characters
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print("File not found. Check the file path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Mounted at /content/drive


In [ ]:
#df = df.drop(columns=['小類'])
print(df)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


                                                 商品名稱      大類      小類
0       PHILIPS 飛利浦 全自動義式咖啡機-EP3246 (金)+湛盧咖啡豆券8張(24包)    家用電器  咖啡機與周邊
1                      PHILIPS 飛利浦 小旋風電動洗鞋機 (GCA1000)    家用電器    其他家電
2          PHILIPS 飛利浦 2021旗艦款八合一乾濕兩用拔刮美體刀 除毛刀 BRE740    美妝保養    除毛器材
3                              PHILIPS 飛利浦 電鬍刀刀頭 RQ11    美妝保養   電動刮鬍刀
4                     PHILIPS 飛利浦 全自動義式咖啡機-EP3246 (金)    家用電器  咖啡機與周邊
...                                               ...     ...     ...
253280            白底青平安扣 贈精美中國繩乙入 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】  愛好與收藏品    原石水晶
253281            Q版可愛牛牛玉墜 天然緬甸硬玉A貨 贈中國繩乙入【文華珠寶翡翠專賣店】  愛好與收藏品    原石水晶
253282       冰種平安扣 起螢放光 購買即贈中國繩乙入 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】  愛好與收藏品    原石水晶
253283            冰種觀音玉墜 贈精美中國繩乙條 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】  愛好與收藏品    原石水晶
253284              冰紫鳳頭如意玉墜 贈中國繩 天然緬甸硬玉A貨【文華珠寶翡翠專賣店】  愛好與收藏品    原石水晶

[253285 rows x 3 columns]


In [ ]:
import torch
import numpy as np
labels = {
    '家用電器': 0,
    '美妝保養': 1,
    '保健': 2,
    '時尚配件': 3,
    '嬰幼兒童裝童鞋': 4,
    '手機平板與周邊': 5,
    '電腦與周邊配件': 6,
    '汽車類': 7,
    '影音': 8,
    '戶外與運動用品': 9,
    '美食、伴手禮': 10,
    '女生包包/精品': 11,
    '男生衣著': 12,
    '居家生活': 13,
    '男生包包': 14,
    '女生衣著': 15,
    '愛好與收藏品': 16,
    '文具、美術用具': 17,
    '母嬰用品': 18,
    '機車類': 19,
    '旅行相關用品/行李箱': 20,
    '相機&空拍機': 21,
    '女鞋': 22,
    '男鞋': 23,
    '寵物': 24,
    '書籍及雜誌期刊': 25,
    '手錶': 26
}



class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['大類']]
        self.texts = [tokenizer(text,
                                padding='max_length',
                                max_length = 512,
                                truncation=True,
                                return_tensors="pt")
                      for text in df['商品名稱']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [ ]:
import numpy as np

np.random.seed(112)
df  =  df.sample(n=9000, random_state=42)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

7200 900 900


In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 27)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練和驗證集
    train, val = Dataset(train_data), Dataset(val_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=3, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=3)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # ------ 驗證模型 -----------
            # 定義兩個變量，用於存儲驗證集的準確率和損失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要計算梯度
            with torch.no_grad():
                # 循環獲取數據集，並用訓練好的模型進行驗證
                for val_input, val_label in val_dataloader:
          # 如果有GPU，則使用GPU，接下來的操作同訓練
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'''Epochs: {epoch_num + 1}
              | Train Loss: {total_loss_train / len(train_data): .3f}
              | Train Accuracy: {total_acc_train / len(train_data): .3f}
              | Val Loss: {total_loss_val / len(val_data): .3f}
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')

訓練時間：45分鐘

In [ ]:
EPOCHS = 4
model = BertClassifier()
LR = 1e-6
train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 480/480 [10:27<00:00,  1.31s/it]


Epochs: 1
              | Train Loss:  0.206
              | Train Accuracy:  0.163
              | Val Loss:  0.184
              | Val Accuracy:  0.301


100%|██████████| 480/480 [10:34<00:00,  1.32s/it]


Epochs: 2
              | Train Loss:  0.164
              | Train Accuracy:  0.419
              | Val Loss:  0.146
              | Val Accuracy:  0.512


100%|██████████| 480/480 [10:25<00:00,  1.30s/it]


Epochs: 3
              | Train Loss:  0.130
              | Train Accuracy:  0.575
              | Val Loss:  0.120
              | Val Accuracy:  0.628


In [ ]:
torch.cuda.empty_cache()

In [ ]:
model_save_path = '/content/drive/Shareddrives/colab/model_state_dict.pth'
torch.save(model.state_dict(), model_save_path)

In [ ]:
model_save_path = '/content/drive/Shareddrives/colab/model_complete.pth'
torch.save(model, model_save_path)

In [ ]:
model = BertClassifier()
model.load_state_dict(torch.load('/content/drive/Shareddrives/colab/model_state_dict.pth'))

<All keys matched successfully>

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)


TypeError: ignored

In [ ]:
jambo = pd.read_csv("/content/drive/Shareddrives/colab/output (10).csv")

In [ ]:

jambo = jambo.rename(columns={'name': '商品名稱'})
jambo = jambo.dropna(subset=['商品名稱'])
jambo = jambo.sample(n=9000, random_state=42)
print(jambo)

       Unnamed: 0                  商品名稱
2310         2310      台灣製不銹鋼鼎鍋玉超夯泡麵碗CM
7614         7614                真善鎂膠囊 
251           251      日本製雞仔牌暖暖鞋墊男/女擇/包
6333         6333                 葉黃素飲 
4884         4884             鍋寶不銹鋼提保鮮盒
...           ...                   ...
10             10         可愛趴趴動物吸水地墊xCM
5885         5885  W PD+QC 多帶線充行動電源 RPP
8883         8883               無痕蓮蓬頭支架
1428         1428              Fila運動長褲
16894       16894       ESSENTIALS燕麥灰短T

[9000 rows x 2 columns]


預處理數據

In [ ]:

from transformers import BertTokenizer
import pandas as pd
import torch
# 示例 DataFrame

# 加载分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 分词和编码
def preprocess(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

# 应用预处理
jambo['encoded'] = jambo['商品名稱'].apply(lambda x: preprocess(x))


创建 DataLoader

In [ ]:
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


# 假设 jambo['encoded'] 是一个列表，其中的每个元素是一个字典
encodings = {'input_ids': [], 'attention_mask': []}

for encoding in jambo['encoded']:
    encodings['input_ids'].append(encoding['input_ids'][0])
    encodings['attention_mask'].append(encoding['attention_mask'][0])

# 创建数据集
dataset = TextDataset(encodings)

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=2)

In [ ]:
model.to('cuda')


predictions = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        output = model(input_ids, attention_mask)
        predictions.append(torch.argmax(output, dim=1))

# 将预测结果转换回 Python 列表
predictions = torch.cat(predictions).cpu().numpy()

# 将预测结果添加到 DataFrame
df['prediction'] = predictions

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder_big = LabelEncoder()

In [ ]:
reverse_labels = {v: k for k, v in labels.items()}



predicted_categories = [reverse_labels[p] for p in predictions]



AttributeError: ignored

**使用bert分大類**


訓練時間:30分鐘

大致上類別分得出來，準確度大概有78成

In [ ]:
jambo['prediction'] = predicted_categories
jambo

,Unnamed: 0,商品名稱,prediction
2310,2310,台灣製不銹鋼鼎鍋玉超夯泡麵碗CM,居家生活
7614,7614,真善鎂膠囊,影音
251,251,日本製雞仔牌暖暖鞋墊男/女擇/包,女生包包/精品
6333,6333,葉黃素飲,書籍及雜誌期刊
4884,4884,鍋寶不銹鋼提保鮮盒,居家生活
...,...,...,...
10,10,可愛趴趴動物吸水地墊xCM,居家生活
5885,5885,W PD+QC 多帶線充行動電源 RPP,電腦與周邊配件
8883,8883,無痕蓮蓬頭支架,居家生活
1428,1428,Fila運動長褲,女生衣著


In [ ]:
from google.colab import files

jambo.to_csv('jambo.csv', encoding = 'utf-8-sig')
files.download('jambo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import re
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
try:

    small_label_df = pd.read_csv("/content/drive/Shareddrives/colab/small_label.csv")
    # Continue processing the content
except UnicodeDecodeError as e:
    # Handle the UnicodeDecodeError by specifying a different encoding or replacing invalid characters
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print("File not found. Check the file path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sub_labels = {}
i = 0
for row in small_label_df["小類"]:
    # You might want to use a more specific attribute of the row as the key, such as an ID
    sub_labels[row] = i  # Using the DataFrame index as the key
    i = i+1
print(sub_labels)

{'咖啡機與周邊': 0, '其他家電': 1, '除毛器材': 2, '電動刮鬍刀': 3, '空氣清淨機': 4, '吹風機': 5, '電動牙刷、配件': 6, '熨斗/掛燙機': 7, '吸塵器/地板保養用具': 8, '其他生活家電': 9, '黃金 / 鉑金 / K金': 10, '項鍊': 11, '純銀': 12, '耳環': 13, '戒指': 14, '手鍊 / 手環': 15, '鑽石': 16, '吊飾 / 吊墜 / 裝飾物': 17, '帽子': 18, '平板保護周邊': 19, '手機保護周邊': 20, '智慧手錶與手環': 21, '其他': 22, '穿戴裝置配件': 23, '螢幕保護貼': 24, '筆電保護膜與保護殼': 25, '充電器': 26, '手機支架': 27, '耳機/耳麥/藍牙耳機': 28, '車用收納': 29, '傳輸線/轉接頭': 30, '燈具': 31, '手機': 32, '美食/電火鍋': 33, '讀卡機': 34, 'USB車充、車載藍芽接收器': 35, '視聽線材/轉換器': 36, '充電座/ 充電盤': 37, '清洗及打蠟用品': 38, '電扇': 39, '雨刷及噴水嘴': 40, '內飾清潔保養用品': 41, '車用芳香及空氣清淨機': 42, '後背包': 43, '手提斜背筆電包': 44, '行動電源': 45, '防水包袋與防水套': 46, '車用頭枕及腰靠墊': 47, '按摩器材、按摩槍': 48, '熟食小吃': 49, '健身器材': 50, '運動頭帶/運動穿戴配件/運動配備': 51, '瑜珈墊': 52, '搖搖杯、運動水壺及其他運動健身用品': 53, '健身及運動護具': 54, '護具': 55, '瑜珈磚/瑜珈圈/瑜珈柱': 56, '重量訓練器材': 57, '保溫杯、保溫瓶': 58, '電腦後背包': 59, 'T恤': 60, '長褲': 61, '裙裝': 62, '洋裝': 63, '細肩帶 / 繞頸背心': 64, '牛仔褲': 65, '手提包': 66, '毛衣 / 針織': 67, '側/肩背包': 68, '襯衫': 69, '吊帶褲 / 吊帶裙': 70, '短褲': 71, '冬季外套 / 大衣': 72, '皮帶

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [sub_labels[label] for label in df['小類']]
        self.texts = [tokenizer(text,
                                padding='max_length',
                                max_length = 512,
                                truncation=True,
                                return_tensors="pt")
                      for text in df['商品名稱']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [ ]:
import numpy as np

np.random.seed(112)
df  =  df.sample(n=9000, random_state=42)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

7200 900 900


In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 632)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):
  # 通過Dataset類獲取訓練和驗證集
    train, val = Dataset(train_data), Dataset(val_data)
    # DataLoader根據batch_size獲取數據，訓練時選擇打亂樣本
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=9, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=9)
  # 判斷是否使用GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    # 定義損失函數和優化器
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()
    # 開始進入訓練循環
    for epoch_num in range(epochs):
      # 定義兩個變量，用於存儲訓練集的準確率和損失
            total_acc_train = 0
            total_loss_train = 0
      # 進度條函數tqdm
            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
        # 通過模型得到輸出
                output = model(input_id, mask)
                # 計算損失
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                # 計算精度
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
        # 模型更新
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # ------ 驗證模型 -----------
            # 定義兩個變量，用於存儲驗證集的準確率和損失
            total_acc_val = 0
            total_loss_val = 0
      # 不需要計算梯度
            with torch.no_grad():
                # 循環獲取數據集，並用訓練好的模型進行驗證
                for val_input, val_label in val_dataloader:
          # 如果有GPU，則使用GPU，接下來的操作同訓練
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'''Epochs: {epoch_num + 1}
              | Train Loss: {total_loss_train / len(train_data): .3f}
              | Train Accuracy: {total_acc_train / len(train_data): .3f}
              | Val Loss: {total_loss_val / len(val_data): .3f}
              | Val Accuracy: {total_acc_val / len(val_data): .3f}''')

In [ ]:
EPOCHS = 3
sub_model=BertClassifier()
LR = 1e-6
train(sub_model, df_train, df_val, LR, EPOCHS)

100%|██████████| 800/800 [11:14<00:00,  1.19it/s]


Epochs: 1
              | Train Loss:  0.707
              | Train Accuracy:  0.036
              | Val Loss:  0.690
              | Val Accuracy:  0.078


100%|██████████| 800/800 [11:15<00:00,  1.18it/s]


Epochs: 2
              | Train Loss:  0.679
              | Train Accuracy:  0.082
              | Val Loss:  0.670
              | Val Accuracy:  0.096


100%|██████████| 800/800 [11:48<00:00,  1.13it/s]


Epochs: 3
              | Train Loss:  0.656
              | Train Accuracy:  0.106
              | Val Loss:  0.646
              | Val Accuracy:  0.124


In [ ]:
path = '/content/drive/Shareddrives/colab/sub_model_state_dict.pth'
torch.save(model.state_dict(), path)

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)
              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(sub_model, df_test)

Test Accuracy:  0.119


In [ ]:
jambo = pd.read_csv("/content/drive/Shareddrives/colab/output (10).csv")

In [ ]:
jambo = jambo.rename(columns={'name': '商品名稱'})
jambo = jambo.dropna(subset=['商品名稱'])
jambo = jambo.sample(n=9000, random_state=42)
print(jambo)

       Unnamed: 0                  商品名稱
2310         2310      台灣製不銹鋼鼎鍋玉超夯泡麵碗CM
7614         7614                真善鎂膠囊 
251           251      日本製雞仔牌暖暖鞋墊男/女擇/包
6333         6333                 葉黃素飲 
4884         4884             鍋寶不銹鋼提保鮮盒
...           ...                   ...
10             10         可愛趴趴動物吸水地墊xCM
5885         5885  W PD+QC 多帶線充行動電源 RPP
8883         8883               無痕蓮蓬頭支架
1428         1428              Fila運動長褲
16894       16894       ESSENTIALS燕麥灰短T

[9000 rows x 2 columns]


In [ ]:
from transformers import BertTokenizer
import pandas as pd
import torch
# 示例 DataFrame

# 加载分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# 分词和编码
def preprocess(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

# 应用预处理
jambo['encoded'] = jambo['商品名稱'].apply(lambda x: preprocess(x))


In [ ]:
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


# 假设 jambo['encoded'] 是一个列表，其中的每个元素是一个字典
encodings = {'input_ids': [], 'attention_mask': []}

for encoding in jambo['encoded']:
    encodings['input_ids'].append(encoding['input_ids'][0])
    encodings['attention_mask'].append(encoding['attention_mask'][0])

# 创建数据集
dataset = TextDataset(encodings)

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=2)

In [ ]:
model.to('cuda')


predictions = []
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        output = sub_model(input_ids, attention_mask)
        predictions.append(torch.argmax(output, dim=1))

# 将预测结果转换回 Python 列表
predictions = torch.cat(predictions).cpu().numpy()

# 将预测结果添加到 DataFrame
jambo['sub_prediction'] = predictions

In [ ]:
reverse_labels = {v: k for k, v in sub_labels.items()}



predicted_categories = [reverse_labels[p] for p in predictions]

**使用bert分小類**


大部分的分類會被分到其他

除了衣服類的比較容易會被bert正確分出來

In [ ]:
jambo['sub_prediction'] = predicted_categories
jambo

,Unnamed: 0,商品名稱,prediction,sub_prediction
2310,2310,台灣製不銹鋼鼎鍋玉超夯泡麵碗CM,居家生活,其他
7614,7614,真善鎂膠囊,影音,其他
251,251,日本製雞仔牌暖暖鞋墊男/女擇/包,女生包包/精品,襪子
6333,6333,葉黃素飲,書籍及雜誌期刊,其他
4884,4884,鍋寶不銹鋼提保鮮盒,居家生活,側/肩背包
...,...,...,...,...
10,10,可愛趴趴動物吸水地墊xCM,居家生活,其他
5885,5885,W PD+QC 多帶線充行動電源 RPP,電腦與周邊配件,其他
8883,8883,無痕蓮蓬頭支架,居家生活,其他
1428,1428,Fila運動長褲,女生衣著,T恤
